In [ ]:
#Testando um alogritmo de deep learning em um conjunto de dados sobre vendas
#Autor: Fernando Schmitt

In [1]:
#criando conexão com o banco de dados

In [2]:
import psycopg2

In [3]:
con = psycopg2.connect(host='localhost', database='fernando', user='postgres', password='******', port=5432 )

In [4]:
#criando cursor para executar consultas
cur = con.cursor()
consulta = 'select * from vendas'
cur.execute(consulta)
registro = cur.fetchall()

In [5]:
#carregando os dados em um dataframe

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(registro, columns=['Id_venda','Id_vendedor', 'Cliente', 'Data', 'Total'])
df.head(110)

,Id_venda,Id_vendedor,Cliente,Data,Total
0,1,1,1,2016-01-01,8053.60
1,2,6,185,2016-01-01,150.40
2,3,7,31,2016-01-02,6087.00
3,4,5,1,2016-01-02,13828.60
4,5,8,120,2016-01-03,26096.66
...,...,...,...,...,...
105,106,4,16,2016-04-01,6771.78
106,107,7,115,2016-04-02,5631.01
107,108,9,27,2016-04-04,207.00
108,109,10,89,2016-04-05,16561.80


In [8]:
#criando modelo de rede neural

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

In [10]:
#Excluindo colunas que não agregam valor na ánalise final e interferem no processamento do modelo
df1 = df.drop(columns=['Data'])


In [11]:
#separando as variaveis x e y em arrays
x = df1.iloc[:, 1:4].values
y = df1.iloc[:, 2].values
x

array([[1, 1, Decimal('8053.60')],
       [6, 185, Decimal('150.40')],
       [7, 31, Decimal('6087.00')],
       ...,
       [1, 238, Decimal('5183.30')],
       [4, 62, Decimal('14592.63')],
       [6, 224, Decimal('18402.00')]], dtype=object)

In [12]:
y

array([  1, 185,  31,   1, 120,  74, 191,   2,  91, 202,   3,  32, 136,
       249,   5, 192,  79, 136, 191, 218, 192,  74,  54, 173,  58,  74,
       233, 191, 249, 218, 249,  79,  58,  74, 233,  89, 192, 202, 233,
        31, 249, 105, 218, 185,  89,  58, 100, 173, 105,  58,   5, 120,
       185, 185, 191, 115,  74, 115,  79, 136, 105,  74, 191, 218,  54,
        27, 191,  31, 105,  58, 120,  74,  89,  32,  91, 202,  89, 100,
       120,  54,  58, 185,  54, 192, 233, 185, 100, 233, 218, 115,  16,
        79, 186,  31, 105,  27,  54,  27, 249, 249, 120,  47,  89, 233,
       249,  16, 115,  27,  89,  31, 191, 218, 218, 191, 202, 100,  58,
        15, 105,  47,  47,  47,  58,  74,  74, 105, 186,  89, 173, 218,
       218, 218, 218, 218, 100,  15, 249,  74, 185, 105, 191, 191, 105,
       192, 192,  31,  15, 115,  16, 191, 105,  47, 100,  15, 120,  91,
        89, 115,  58, 249, 186,  89, 173, 249,  58,  54, 120, 100, 249,
       120, 115, 115,  58,  47, 185,  91, 233, 191,  89, 192, 11

In [13]:
lben = LabelEncoder()
x[:, 2] = lben.fit_transform(x[:, 2])
x

array([[1, 1, 188],
       [6, 185, 16],
       [7, 31, 146],
       ...,
       [1, 238, 126],
       [4, 62, 272],
       [6, 224, 303]], dtype=object)

In [14]:
lbc = LabelEncoder()
y = lbc.fit_transform(y)
y

array([  0,  99,  16,   0,  66,  41, 104,   1,  50, 110,   2,  17,  73,
       135,   3, 105,  45,  73, 104, 116, 105,  41,  31,  92,  33,  41,
       125, 104, 135, 116, 135,  45,  33,  41, 125,  49, 105, 110, 125,
        16, 135,  57, 116,  99,  49,  33,  53,  92,  57,  33,   3,  66,
        99,  99, 104,  63,  41,  63,  45,  73,  57,  41, 104, 116,  31,
        14, 104,  16,  57,  33,  66,  41,  49,  17,  50, 110,  49,  53,
        66,  31,  33,  99,  31, 105, 125,  99,  53, 125, 116,  63,   7,
        45, 100,  16,  57,  14,  31,  14, 135, 135,  66,  26,  49, 125,
       135,   7,  63,  14,  49,  16, 104, 116, 116, 104, 110,  53,  33,
         6,  57,  26,  26,  26,  33,  41,  41,  57, 100,  49,  92, 116,
       116, 116, 116, 116,  53,   6, 135,  41,  99,  57, 104, 104,  57,
       105, 105,  16,   6,  63,   7, 104,  57,  26,  53,   6,  66,  50,
        49,  63,  33, 135, 100,  49,  92, 135,  33,  31,  66,  53, 135,
        66,  63,  63,  33,  26,  99,  50, 125, 104,  49, 105,  6

In [15]:
print(type(x))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [16]:
#treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=0)
print(len(x_treino), len(x_teste), len(y_treino), len(y_teste))

320 80 320 80


In [17]:
print(y_treino)

[124  31  63  63  47  99  33   3  55 107 112   1  41  50  26   6   6  18
  30  24  88 116  12 135  75  44  99  40   6 127  31  63  29  41  98  41
  43  33   7  51 100  99 125  17   4  57  31  59   8 135  57  27 104  63
  31  26   9  45  96  73  33  99 135 110 101 125 132  99  16  33  11  26
  85 116  53 135  20 100  17  77  22 116  41  42  92 125  50  14  99  41
  53 131  66  86  99  42  49  53  56  37  45  17  41   0  17  17 116 105
  90 115  66  82  65 104  16  16  49  53 109 116  99  99  16  25 128 120
 125  49  73  89  16  33  52  49  45 116  16 116  66 128  58 123 135 104
  28  31  45  24  41 135  87 116  21  81  53 100 135  49 105  66 128  57
  46 116  16  95 104 104  63  53  45  84 110  16  39  99   5  57  99 111
  63  19 116 125  42   0  25  57  53  63 111 119 111  16 122  33  88  17
  83  53  59 105 121  45  57  70   3  53  23  72  68  41  45 104 126 116
  63  92  99   0  63  78 135  68  49 102  57 127  14 118  66  33  20  57
 116 124  49   7 105  63 116  54  35  96  10  14 12

In [20]:
sc = StandardScaler()
x_treino = sc.fit_transform(x_treino)
x_teste = sc.transform(x_teste)
x_teste

array([[-0.80802587,  1.27590567, -1.30983138],
       [-1.14558237, -0.54445151,  1.09586796],
       [ 0.20464363,  1.58835504,  0.08129041],
       [-1.14558237,  0.16195575,  1.74436082],
       [-0.13291287,  0.82760875, -1.60269913],
       [-1.14558237, -0.25917166,  0.37415816],
       [ 0.54220013, -0.44935823,  1.56654826],
       [-0.80802587,  0.77326973, -1.59223956],
       [-0.13291287,  0.90911728, -0.28479427],
       [-1.14558237,  1.27590567, -0.42076858],
       [-1.14558237,  1.05854959,  0.55197072],
       [ 1.55486962, -0.32709543, -0.72409589],
       [-0.13291287,  0.90911728,  1.5560887 ],
       [ 1.55486962,  0.51515939, -1.15293795],
       [-0.47046937,  0.16195575,  1.33643789],
       [-0.47046937,  0.16195575,  0.2800221 ],
       [-1.14558237, -0.16407837,  0.99127234],
       [ 1.55486962,  0.90911728,  1.0017319 ],
       [ 0.20464363, -0.12332411, -1.06926145],
       [-0.80802587, -1.04708745,  0.25910297],
       [ 1.55486962,  1.479677  , -0.588

In [21]:
classificador = Sequential()
classificador.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu', input_dim = 3))
classificador.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
classificador.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classificador.fit(x_treino, y_treino, batch_size = 10, epochs = 2000)
                        

Epoch 1/2000
32/32 [==============================] - 0s 1ms/step - loss: -0.4809 - accuracy: 0.0031
Epoch 2/2000
32/32 [==============================] - 0s 1ms/step - loss: -3.7155 - accuracy: 0.0031
Epoch 3/2000
32/32 [==============================] - 0s 2ms/step - loss: -9.4248 - accuracy: 0.0031
Epoch 4/2000
32/32 [==============================] - 0s 2ms/step - loss: -19.8162 - accuracy: 0.0031
Epoch 5/2000
32/32 [==============================] - 0s 1ms/step - loss: -36.9340 - accuracy: 0.0031
Epoch 6/2000
32/32 [==============================] - 0s 1ms/step - loss: -63.9107 - accuracy: 0.0031
Epoch 7/2000
32/32 [==============================] - 0s 1ms/step - loss: -103.4369 - accuracy: 0.0031
Epoch 8/2000
32/32 [==============================] - 0s 1ms/step - loss: -157.7994 - accuracy: 0.0031
Epoch 9/2000
32/32 [==============================] - 0s 1ms/step - loss: -231.1778 - accuracy: 0.0031
Epoch 10/2000
32/32 [==============================] - 0s 1ms/step - loss: -324.02

32/32 [==============================] - 0s 1ms/step - loss: -131247.5000 - accuracy: 0.0031
Epoch 79/2000
32/32 [==============================] - 0s 1ms/step - loss: -135641.9062 - accuracy: 0.0031
Epoch 80/2000
32/32 [==============================] - 0s 2ms/step - loss: -140150.4844 - accuracy: 0.0031
Epoch 81/2000
32/32 [==============================] - 0s 2ms/step - loss: -144705.4219 - accuracy: 0.0031
Epoch 82/2000
32/32 [==============================] - 0s 2ms/step - loss: -149426.7188 - accuracy: 0.0031
Epoch 83/2000
32/32 [==============================] - 0s 1ms/step - loss: -154105.5938 - accuracy: 0.0031
Epoch 84/2000
32/32 [==============================] - 0s 1ms/step - loss: -158981.2031 - accuracy: 0.0031
Epoch 85/2000
32/32 [==============================] - 0s 2ms/step - loss: -163815.6562 - accuracy: 0.0031
Epoch 86/2000
32/32 [==============================] - 0s 1ms/step - loss: -168923.6094 - accuracy: 0.0031
Epoch 87/2000
32/32 [==============================

32/32 [==============================] - 0s 2ms/step - loss: -725025.6875 - accuracy: 0.0031
Epoch 155/2000
32/32 [==============================] - 0s 1ms/step - loss: -736972.0625 - accuracy: 0.0031
Epoch 156/2000
32/32 [==============================] - 0s 1ms/step - loss: -748427.5000 - accuracy: 0.0031
Epoch 157/2000
32/32 [==============================] - 0s 1ms/step - loss: -760139.4375 - accuracy: 0.0031 
Epoch 158/2000
32/32 [==============================] - 0s 2ms/step - loss: -772127.8750 - accuracy: 0.0031
Epoch 159/2000
32/32 [==============================] - 0s 1ms/step - loss: -784184.6875 - accuracy: 0.0031
Epoch 160/2000
32/32 [==============================] - 0s 1ms/step - loss: -796161.4375 - accuracy: 0.0031
Epoch 161/2000
32/32 [==============================] - 0s 1ms/step - loss: -808363.3750 - accuracy: 0.0031
Epoch 162/2000
32/32 [==============================] - 0s 2ms/step - loss: -820683.8750 - accuracy: 0.0031 
Epoch 163/2000
32/32 [===================

32/32 [==============================] - 0s 1ms/step - loss: -1909206.0000 - accuracy: 0.0031
Epoch 230/2000
32/32 [==============================] - 0s 1ms/step - loss: -1929602.3750 - accuracy: 0.0031
Epoch 231/2000
32/32 [==============================] - 0s 1ms/step - loss: -1950302.6250 - accuracy: 0.0031
Epoch 232/2000
32/32 [==============================] - 0s 1ms/step - loss: -1970685.6250 - accuracy: 0.0031
Epoch 233/2000
32/32 [==============================] - 0s 1ms/step - loss: -1991595.3750 - accuracy: 0.0031
Epoch 234/2000
32/32 [==============================] - 0s 1ms/step - loss: -2012630.3750 - accuracy: 0.0031
Epoch 235/2000
32/32 [==============================] - 0s 1ms/step - loss: -2033230.6250 - accuracy: 0.0031
Epoch 236/2000
32/32 [==============================] - 0s 1ms/step - loss: -2054701.0000 - accuracy: 0.0031
Epoch 237/2000
32/32 [==============================] - 0s 1ms/step - loss: -2076211.0000 - accuracy: 0.0031
Epoch 238/2000
32/32 [============

32/32 [==============================] - 0s 1ms/step - loss: -3820406.7500 - accuracy: 0.0031
Epoch 305/2000
32/32 [==============================] - 0s 1ms/step - loss: -3851313.2500 - accuracy: 0.0031
Epoch 306/2000
32/32 [==============================] - 0s 1ms/step - loss: -3882678.5000 - accuracy: 0.0031
Epoch 307/2000
32/32 [==============================] - 0s 2ms/step - loss: -3913953.5000 - accuracy: 0.0031
Epoch 308/2000
32/32 [==============================] - 0s 1ms/step - loss: -3945594.7500 - accuracy: 0.0031
Epoch 309/2000
32/32 [==============================] - 0s 1ms/step - loss: -3976998.5000 - accuracy: 0.0031
Epoch 310/2000
32/32 [==============================] - 0s 1ms/step - loss: -4009154.5000 - accuracy: 0.0031
Epoch 311/2000
32/32 [==============================] - 0s 1ms/step - loss: -4041008.0000 - accuracy: 0.0031
Epoch 312/2000
32/32 [==============================] - 0s 1ms/step - loss: -4072821.5000 - accuracy: 0.0031
Epoch 313/2000
32/32 [============

...

32/32 [==============================] - 0s 1ms/step - loss: -335575680.0000 - accuracy: 0.0031
Epoch 1704/2000
32/32 [==============================] - 0s 1ms/step - loss: -336109760.0000 - accuracy: 0.0031
Epoch 1705/2000
32/32 [==============================] - 0s 1ms/step - loss: -336657760.0000 - accuracy: 0.0031
Epoch 1706/2000
32/32 [==============================] - 0s 1ms/step - loss: -337204800.0000 - accuracy: 0.0031
Epoch 1707/2000
32/32 [==============================] - 0s 1ms/step - loss: -337732544.0000 - accuracy: 0.0031
Epoch 1708/2000
32/32 [==============================] - 0s 1ms/step - loss: -338282336.0000 - accuracy: 0.0031
Epoch 1709/2000
32/32 [==============================] - 0s 1ms/step - loss: -338829504.0000 - accuracy: 0.0031
Epoch 1710/2000
32/32 [==============================] - 0s 1ms/step - loss: -339363488.0000 - accuracy: 0.0031
Epoch 1711/2000
32/32 [==============================] - 0s 1ms/step - loss: -339910592.0000 - accuracy: 0.0031
Epoch 17

32/32 [==============================] - 0s 1ms/step - loss: -376601440.0000 - accuracy: 0.0031
Epoch 1777/2000
32/32 [==============================] - 0s 1ms/step - loss: -377174624.0000 - accuracy: 0.0031
Epoch 1778/2000
32/32 [==============================] - 0s 1ms/step - loss: -377765120.0000 - accuracy: 0.0031
Epoch 1779/2000
32/32 [==============================] - 0s 1ms/step - loss: -378351552.0000 - accuracy: 0.0031
Epoch 1780/2000
32/32 [==============================] - 0s 1ms/step - loss: -378943424.0000 - accuracy: 0.0031
Epoch 1781/2000
32/32 [==============================] - 0s 1ms/step - loss: -379518400.0000 - accuracy: 0.0031
Epoch 1782/2000
32/32 [==============================] - 0s 1ms/step - loss: -380114816.0000 - accuracy: 0.0031
Epoch 1783/2000
32/32 [==============================] - 0s 1ms/step - loss: -380699072.0000 - accuracy: 0.0031
Epoch 1784/2000
32/32 [==============================] - 0s 1ms/step - loss: -381292640.0000 - accuracy: 0.0031
Epoch 17

32/32 [==============================] - 0s 1ms/step - loss: -420860352.0000 - accuracy: 0.0031
Epoch 1850/2000
32/32 [==============================] - 0s 1ms/step - loss: -421487328.0000 - accuracy: 0.0031
Epoch 1851/2000
32/32 [==============================] - 0s 1ms/step - loss: -422119104.0000 - accuracy: 0.0031
Epoch 1852/2000
32/32 [==============================] - 0s 1ms/step - loss: -422746176.0000 - accuracy: 0.0031
Epoch 1853/2000
32/32 [==============================] - 0s 1ms/step - loss: -423378240.0000 - accuracy: 0.0031
Epoch 1854/2000
32/32 [==============================] - 0s 1ms/step - loss: -424010816.0000 - accuracy: 0.0031
Epoch 1855/2000
32/32 [==============================] - 0s 1ms/step - loss: -424638880.0000 - accuracy: 0.0031
Epoch 1856/2000
32/32 [==============================] - 0s 1ms/step - loss: -425273632.0000 - accuracy: 0.0031
Epoch 1857/2000
32/32 [==============================] - 0s 2ms/step - loss: -425904832.0000 - accuracy: 0.0031
Epoch 18

32/32 [==============================] - 0s 1ms/step - loss: -468364896.0000 - accuracy: 0.0031
Epoch 1923/2000
32/32 [==============================] - 0s 1ms/step - loss: -469039424.0000 - accuracy: 0.0031
Epoch 1924/2000
32/32 [==============================] - 0s 1ms/step - loss: -469717760.0000 - accuracy: 0.0031
Epoch 1925/2000
32/32 [==============================] - 0s 1ms/step - loss: -470384992.0000 - accuracy: 0.0031
Epoch 1926/2000
32/32 [==============================] - 0s 1ms/step - loss: -471070400.0000 - accuracy: 0.0031
Epoch 1927/2000
32/32 [==============================] - 0s 1ms/step - loss: -471745184.0000 - accuracy: 0.0031
Epoch 1928/2000
32/32 [==============================] - 0s 1ms/step - loss: -472421376.0000 - accuracy: 0.0031
Epoch 1929/2000
32/32 [==============================] - 0s 1ms/step - loss: -473094400.0000 - accuracy: 0.0031
Epoch 1930/2000
32/32 [==============================] - 0s 1ms/step - loss: -473782272.0000 - accuracy: 0.0031
Epoch 19

32/32 [==============================] - 0s 1ms/step - loss: -519097504.0000 - accuracy: 0.0031
Epoch 1996/2000
32/32 [==============================] - 0s 1ms/step - loss: -519816000.0000 - accuracy: 0.0031
Epoch 1997/2000
32/32 [==============================] - 0s 1ms/step - loss: -520535808.0000 - accuracy: 0.0031
Epoch 1998/2000
32/32 [==============================] - 0s 1ms/step - loss: -521259456.0000 - accuracy: 0.0031
Epoch 1999/2000
32/32 [==============================] - 0s 1ms/step - loss: -521975296.0000 - accuracy: 0.0031
Epoch 2000/2000
32/32 [==============================] - 0s 1ms/step - loss: -522698304.0000 - accuracy: 0.0031


In [22]:
#
pred = classificador.predict(x_teste)
pred = (pred > 0.5)
pred

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [23]:
#matriz de confusão com acertos e erros
cmx = confusion_matrix(y_teste,pred)
cmx

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [24]:
#taxa de acerto do modelo
txa = accuracy_score(y_teste, pred)
txe = 1 - txa
txa

0.0125

In [25]:
txe

0.9875